# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [21]:
#Your code here
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import time

api_key = 'Y06pnYNUNzVXizQq0PA1hxAxJjKqo8TgGl_H4l5d8n9vb2AYKQb8w5xehOOoqltKFeF0Xxk1t8ayva7a4eaXxsu3rG0K-MlwJUr6ICaraJMTdSlC9FLtjaATGLGSXHYx'


host = 'https://api.yelp.com'
path = '/v3/businesses/search'

#Search Terms
term = 'cocktails'
location = 'Brooklyn, NY'

#Set search_limit to '1' to find Total Results
search_limit = 50
max_search = 5000

headers = {
        'Authorization': 'Bearer {}'.format(api_key) 
    }

#Build the Request Syntax
def request(api_key, host, path, url_params=None):
    
    url_params = url_params or {}
    url = '{}{}'.format(host, path)

    response = requests.get(url, headers=headers, params=url_params)
    
    return response.json()

#Build the Search Syntax
def search(api_key, term, offset_index, location):
    
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': search_limit,
        'offset': offset_index,
        'price': '2, 3, 4'
    }
    
    return request(api_key, host, path, url_params=url_params)




## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [73]:
def make_api_calls(api_key, term, location):
    all_businesses = []
    offset = search_limit
    calls = 20

    for i in range(0, offset*calls, offset):
        x = search(api_key, term, i, location)
        for business in x['businesses']:
            all_businesses.append(business)
        time.sleep(1)
        
    return all_businesses

response = make_api_calls(api_key, term, location)

In [74]:
response[0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [75]:
df = pd.DataFrame(response)
df['price'] = [len(str(price)) for price in df['price']]

df['value'] = df['rating'] / df['price']

value_sort = df.sort_values(by=['value'], ascending=False)
value_sort = value_sort.loc[df['review_count'] > 10]
value_sort.head(10)

df.describe()

,distance,price,rating,review_count,value
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,6439.335560,2.156000,4.03750,221.893000,1.917125
std,3352.376360,0.389635,0.43936,278.604095,0.324212
min,576.394172,2.000000,1.00000,1.000000,0.333333
25%,3641.166917,2.000000,4.00000,58.000000,1.750000
50%,6385.747385,2.000000,4.00000,134.000000,2.000000
75%,8155.396277,2.000000,4.50000,287.250000,2.000000
max,23344.596465,4.000000,5.00000,2771.000000,2.500000


In [104]:
df['latitude'] = [coordinate['latitude'] for coordinate in df['coordinates']]
df['longitude'] = [coordinate['longitude'] for coordinate in df['coordinates']]
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,value,latitude,longitude
0,sweet-polly-brooklyn,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 40.68047, 'longitude': -73.9744599}",(718) 484-9600,2484.264262,0itsOZxeHy2KPnA40Rwexw,https://s3-media4.fl.yelpcdn.com/bphoto/n7Caqu...,False,"{'address1': '71 6th Ave', 'address2': '', 'ad...",Sweet Polly,+17184849600,2,4.5,108,[],https://www.yelp.com/biz/sweet-polly-brooklyn?...,2.25,40.680470,-73.974460
1,yours-sincerely-brooklyn,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 40.7028639, 'longitude': -73.9291...",,5878.879081,-9ENCGNO_dCsFlzho6KSDA,https://s3-media2.fl.yelpcdn.com/bphoto/atWgeF...,False,"{'address1': '41 Wilson Ave', 'address2': '', ...",Yours Sincerely,,2,4.5,58,[],https://www.yelp.com/biz/yours-sincerely-brook...,2.25,40.702864,-73.929150
2,blueprint-brooklyn-2,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 40.6769, 'longitude': -73.980411}",(718) 622-6644,2295.882527,XLT7aNLJEhIdoieNjPw8Gg,https://s3-media1.fl.yelpcdn.com/bphoto/xJSV7S...,False,"{'address1': '196 5th Ave', 'address2': '', 'a...",Blueprint,+17186226644,2,4.5,312,[],https://www.yelp.com/biz/blueprint-brooklyn-2?...,2.25,40.676900,-73.980411
3,dirty-precious-brooklyn,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 40.6763896691334, 'longitude': -7...",(646) 685-4481,2546.401167,-WUCua_aSIpfsHKMFlMEMg,https://s3-media2.fl.yelpcdn.com/bphoto/NJDSQR...,False,"{'address1': '317 3rd Ave', 'address2': '', 'a...",Dirty Precious,+16466854481,2,4.0,65,[],https://www.yelp.com/biz/dirty-precious-brookl...,2.00,40.676390,-73.986717
4,townhouse-275-brooklyn,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 40.6182556, 'longitude': -74.033989}",(347) 560-4001,7170.293142,2o3xF6DnHDfZ3k1mPioh4A,https://s3-media2.fl.yelpcdn.com/bphoto/7IcU7x...,False,"{'address1': '275 94th St', 'address2': None, ...",Townhouse 275,+13475604001,2,4.5,56,[],https://www.yelp.com/biz/townhouse-275-brookly...,2.25,40.618256,-74.033989


In [106]:
jupiter = df.loc[df['name'] == "Jupiter Disco"]
jupiter_lat = float(jupiter.get('latitude'))
jupiter_long = float(jupiter.get('longitude'))
jupiter_lat

40.70814

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here
import folium

#Center map on Jupiter Disco
lat = jupiter_lat
long = jupiter_long

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=13)
base_map

#Points
x = [float(latitude) for latitude in df['latitude']]
y = [float(longitude) for longitude in df['longitude']]

points = list(zip(x, y))
for p in points:
    lat = p[0]
    long = p[1]
    popup_text = "Lattitude: {}, Longitude: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)

base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!